# CycleGAN for Document Denoising

## Set up the input pipeline

Install the [tensorflow_examples](https://github.com/tensorflow/examples) package that enables importing of the generator and the discriminator.

In [3]:
!pip install git+https://github.com/tensorflow/examples.git

  Cloning https://github.com/tensorflow/examples.git to /tmp/pip-req-build-jng76rt3
  Running command git clone --filter=blob:none --quiet https://github.com/tensorflow/examples.git /tmp/pip-req-build-jng76rt3
  Resolved https://github.com/tensorflow/examples.git to commit fff4bcda7201645a1efaea4534403daf5fc03d42
  Preparing metadata (setup.py) ... done


## Import libraries and data

In [4]:
import tensorflow as tf
from tensorflow_examples.models.pix2pix import pix2pix

import os
import cv2
import numpy as np
from PIL import Image

AUTOTUNE = tf.data.AUTOTUNE

2024-01-27 22:50:48.658522: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-27 22:50:48.712379: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-27 22:50:48.712429: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-27 22:50:48.713990: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-27 22:50:48.722454: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-27 22:50:48.723596: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
# check GPU details
!nvidia-smi

Sat Jan 27 02:09:17 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.199.02   Driver Version: 470.199.02   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:05.0 Off |                  Off |
| N/A   40C    P0    38W / 300W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
# the whole data path
path = '/data/ephemeral/home/data/medical/img/'
# the directory storing images to be processed
to_process_path = 'train/'
# the directory storing processed images
processed_path = 'processed_train/'
# list storing image filenames
to_process_img = sorted(os.listdir(path + to_process_path))

## Data preparation
Next step is to define function to process images and then store this images in list. As there is not as many data, we do not need to work in batches.

In [6]:
IMG_WIDTH = 3072
IMG_HEIGHT = 4096

# prepare function
def process_image(path):
    img = cv2.imread(path)
    img = np.asarray(img, dtype="float32")
    original_size = img.shape[:2]
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    img = img/255.0
    img = np.reshape(img, (IMG_HEIGHT, IMG_WIDTH, 3))
    return img, original_size

In [7]:
# preprocess images
chinese_invoice = []
original_sizes = []
image_files = [path + to_process_path + f for f in to_process_img]


for f in image_files:
    img, orig_size = process_image(f)
    chinese_invoice.append(img)
    original_sizes.append(orig_size)

chinese_invoice = np.asarray(chinese_invoice)

## Import and reuse the Pix2Pix models

In [8]:
OUTPUT_CHANNELS = 3

generator_g = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')
generator_f = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')

discriminator_x = pix2pix.discriminator(norm_type='instancenorm', target=False)
discriminator_y = pix2pix.discriminator(norm_type='instancenorm', target=False)

2024-01-27 22:53:29.755523: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-27 22:53:29.756999: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Initialize the optimizers for all the generators and the discriminators.

In [9]:
generator_g_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
generator_f_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

discriminator_x_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_y_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

## Checkpoints

In [10]:
checkpoint_path = "/data/ephemeral/home/DocumentDenoise/checkpoints/cycleGAN"

ckpt = tf.train.Checkpoint(generator_g=generator_g,
                           generator_f=generator_f,
                           discriminator_x=discriminator_x,
                           discriminator_y=discriminator_y,
                           generator_g_optimizer=generator_g_optimizer,
                           generator_f_optimizer=generator_f_optimizer,
                           discriminator_x_optimizer=discriminator_x_optimizer,
                           discriminator_y_optimizer=discriminator_y_optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print ('Latest checkpoint restored!!')

Latest checkpoint restored!!


## Denoising and Save images

In [11]:
def tensor_to_image(tensor):
    tensor = tensor*255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor)>3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]
    return Image.fromarray(tensor)

In [13]:
i = 0
for m, (image, orig_size) in enumerate(zip(chinese_invoice, original_sizes)) :
  prediction = generator_g(image.reshape(1,IMG_HEIGHT, IMG_WIDTH,3))
  im_path = path + processed_path + to_process_img[i]
  im = tensor_to_image(prediction)
  im_np = np.array(im)
  orig_height, orig_width = orig_size
  im_resized = cv2.resize(im_np, (orig_width, orig_height))
  im_final = Image.fromarray(im_resized)
  im_final.save(im_path)
  i += 1

## Next steps
- Training the model on a larger dataset
- Tuning parameters to achieve greater performance
- Fine-tuning the models on a different dataset to implement more functions (e.g., watermark removal and motion deblur)